In [18]:
import Pkg
Pkg.activate("..")

  Activating project at `~/Projects/latentplan.jl`


In [19]:
using ArgParse: ArgParseSettings, @add_arg_table!, parse_args
using Statistics: mean
using Printf
using Knet
using Debugger: @enter, @bp, @run
using JSON
using PyCall

include("LPCore.jl")
include("setup.jl")

s = ArgParseSettings()
@add_arg_table! s begin
    "--dataset"
        help = "which environment to use"
        arg_type = String
        default = "halfcheetah-medium-expert-v2"
    "--exp_name"
        help = "name of the experiment"
        arg_type = String
        default = "debug"
    "--seed"
        help = "seed"
        default = 42
    "--beam_width"
        default = 64
    "--n_expand"
        default = 4
    "--suffix"
        default = ""
    "--config"
        help = "relative jl file path with configurations"
        arg_type = String
        default = "../config/vqvae.jl"
end;

In [20]:
@pyimport torch
@pyimport numpy

weights = torch.load("test/files/gpt_trained.pt", map_location=torch.device("cpu"))
prior_weights = torch.load("test/files/prior_model.pt");

Dict{Any, Any} with 75 entries:
  "blocks.2.ln2.bias"          => PyObject tensor([0., 0., 0., 0., 0., 0., 0., …
  "blocks.0.attn.value.bias"   => PyObject tensor([0., 0., 0., 0., 0., 0., 0., …
  "state_emb.bias"             => PyObject tensor([0., 0., 0., 0., 0., 0., 0., …
  "blocks.1.attn.proj.bias"    => PyObject tensor([0., 0., 0., 0., 0., 0., 0., …
  "blocks.3.attn.value.bias"   => PyObject tensor([0., 0., 0., 0., 0., 0., 0., …
  "blocks.0.ln2.bias"          => PyObject tensor([0., 0., 0., 0., 0., 0., 0., …
  "blocks.0.mlp.0.bias"        => PyObject tensor([0., 0., 0.,  ..., 0., 0., 0.…
  "blocks.3.ln1.weight"        => PyObject tensor([1., 1., 1., 1., 1., 1., 1., …
  "pos_emb"                    => PyObject tensor([[[0., 0., 0.,  ..., 0., 0., …
  "blocks.0.mlp.0.weight"      => PyObject tensor([[ 0.0037,  0.0118, -0.0088, …
  "blocks.2.attn.query.bias"   => PyObject tensor([0., 0., 0., 0., 0., 0., 0., …
  "blocks.3.attn.mask"         => PyObject tensor([[[[1., 0., 0., 0., 0., 0.,

In [13]:
#######################
####### setup ########
#######################

super_args = Dict{String, Any}(
    "dataset"=> "hopper-medium-replay-v2",
    "exp_name"=> "T-1-42",
    "seed"=> 42,
    "config"=> "../config/vqvae.jl",
    "beam_width"=>64,
    "n_expand"=>4,
    "suffix"=>""
)
args = parser(super_args, experiment="plan")

args["logbase"] = expanduser(args["logbase"])
args["savepath"] = expanduser(args["savepath"])
args["loadpath"] = joinpath(args["logbase"], args["dataset"], args["exp_name"])

[ utils/setup ] Reading config: ../config/vqvae.jl:hopper_medium_replay_v2
/Users/mehmeteneserciyes/logs_julia/hopper-medium-replay-v2/T-1-42/ already exists. Proceeding...
Made directory /Users/mehmeteneserciyes/logs_julia/hopper-medium-replay-v2/T-1-42/


"/Users/mehmeteneserciyes/logs_julia/hopper-medium-replay-v2/T-1-42"

In [9]:
env = load_environment(args["dataset"])
dataset_config = Knet.load(joinpath(args["loadpath"] , "dataset_config.jld2"), "config")

dataset = SequenceDataset(
    dataset_config["env_name"];
    penalty=dataset_config["penalty"],
    sequence_length=dataset_config["sequence_length"], 
    step=dataset_config["step"], 
    discount=dataset_config["discount"], 
    disable_goal=dataset_config["disable_goal"], 
    normalize_raw=dataset_config["normalize_raw"], 
    normalize_reward=dataset_config["normalize_reward"],
    max_path_length=dataset_config["max_path_length"],
    atype=dataset_config["atype"]
)

[ datasets/sequence ] Sequence length: 25 | Step: 1 | Max path length: 1000
[ datasets/sequence ] Loading...
✓
[ datasets/sequence ] Segmenting...

load datafile: 100%|███████████████████████████| 11/11 [00:00<00:00, 14.20it/s]



✓


Calculating values 100%|█████████████████████████████████| Time: 0:00:04


SequenceDataset(PyObject <d4rl.utils.wrappers.NormalizedBoxEnv object at 0x157f17e80>, 25, 1, 1000, "cuda:0", false, true, true, Float32[1.2305137; -0.043713972; … ; -0.14465895; -0.19652691;;], Float32[0.17565039; 0.0636919; … ; 1.6540343; 5.1086063;;], Float32[0.09147097; 0.37909505; -0.00019374755;;], Float32[0.53679484; 0.5815471; 0.75734293;;], 2.372544f0, 1.0507907f0, Float32[0.110015035 0.10879681 … 0.5080938 0.40982184; 0.7504013 0.714008 … -0.77624345 -0.819888; … ; 0.08837258 0.08248433 … 0.39702943 0.17106935; 0.03892147 -0.22365776 … 0.033442777 0.035735335], Float32[-0.98633546 -1.2569964 … -1.3178613 0.85888374; -0.014222167 -0.42894647 … 1.0531265 1.052914; -1.2321614 -0.9727596 … 1.2837203 1.2837336], Float32[0.110015035 0.10879681 … 0.5080938 0.40982184; 0.7504013 0.714008 … -0.77624345 -0.819888; … ; -1.3354627 -1.3966663 … 0.21831672 0.22489393; -2.1312575 -2.1479235 … -1.3035873 -1.3279964], Float32[-1.3354627 -1.3966663 … 0.21831672 0.22489393], Bool[0, 0, 0, 0, 0,

# Representation model init and weight loading

In [15]:
model_config = Knet.load(joinpath("/Users/mehmeteneserciyes/logs_julia/hopper-medium-replay-v2/T-1-42", "model_config.jld2"), "config")

OrderedCollections.OrderedDict{String, Any} with 52 entries:
  "model"                      => "VQTransformer"
  "tag"                        => "experiment"
  "state_conditional"          => true
  "N"                          => 100
  "discount"                   => 0.99
  "n_layer"                    => 4
  "n_head"                     => 4
  "n_epochs_ref"               => 50
  "n_saves"                    => 3
  "logbase"                    => "~/logs_julia/"
  "device"                     => "cuda"
  "K"                          => 512
  "latent_step"                => 3
  "n_embd"                     => 512
  "trajectory_embd"            => 512
  "batch_size"                 => 512
  "learning_rate"              => 0.0002
  "lr_decay"                   => false
  "seed"                       => 42
  "embd_pdrop"                 => 0.0
  "resid_pdrop"                => 0.0
  "attn_pdrop"                 => 0.0
  "step"                       => 1
  "subsampled_sequence_length" => 

In [16]:
gpt = VQContinuousVAE(model_config);

In [21]:
# encoder
gpt.model.embed.w = Param(atype(weights["model.embed.weight"][:cpu]()[:numpy]()))
gpt.model.embed.b = Param(atype(weights["model.embed.bias"][:cpu]()[:numpy]()))

gpt.model.pos_emb = Param(atype(permutedims(weights["model.pos_emb"][:cpu]()[:numpy](), (3,2,1))))

for i in 1:model_config["n_layer"]
    gpt.model.encoder.layers[i].ln1.a = Param(atype(weights["model.encoder.$(i-1).ln1.weight"][:cpu]()[:numpy]()))
    gpt.model.encoder.layers[i].ln1.b = Param(atype(weights["model.encoder.$(i-1).ln1.bias"][:cpu]()[:numpy]()))
    gpt.model.encoder.layers[i].ln2.a = Param(atype(weights["model.encoder.$(i-1).ln2.weight"][:cpu]()[:numpy]()))
    gpt.model.encoder.layers[i].ln2.b = Param(atype(weights["model.encoder.$(i-1).ln2.bias"][:cpu]()[:numpy]()))

    gpt.model.encoder.layers[i].attn.key.w = Param(atype(weights["model.encoder.$(i-1).attn.key.weight"][:cpu]()[:numpy]()))
    gpt.model.encoder.layers[i].attn.key.b = Param(atype(weights["model.encoder.$(i-1).attn.key.bias"][:cpu]()[:numpy]()))
    gpt.model.encoder.layers[i].attn.query.w = Param(atype(weights["model.encoder.$(i-1).attn.query.weight"][:cpu]()[:numpy]()))
    gpt.model.encoder.layers[i].attn.query.b = Param(atype(weights["model.encoder.$(i-1).attn.query.bias"][:cpu]()[:numpy]()))
    gpt.model.encoder.layers[i].attn.value.w = Param(atype(weights["model.encoder.$(i-1).attn.value.weight"][:cpu]()[:numpy]()))
    gpt.model.encoder.layers[i].attn.value.b = Param(atype(weights["model.encoder.$(i-1).attn.value.bias"][:cpu]()[:numpy]()))
    gpt.model.encoder.layers[i].attn.proj.w = Param(atype(weights["model.encoder.$(i-1).attn.proj.weight"][:cpu]()[:numpy]()))
    gpt.model.encoder.layers[i].attn.proj.b = Param(atype(weights["model.encoder.$(i-1).attn.proj.bias"][:cpu]()[:numpy]()))

    gpt.model.encoder.layers[i].mlp.layers[1].w = Param(atype(weights["model.encoder.$(i-1).mlp.0.weight"][:cpu]()[:numpy]()))
    gpt.model.encoder.layers[i].mlp.layers[1].b = Param(atype(weights["model.encoder.$(i-1).mlp.0.bias"][:cpu]()[:numpy]()))
    gpt.model.encoder.layers[i].mlp.layers[3].w = Param(atype(weights["model.encoder.$(i-1).mlp.2.weight"][:cpu]()[:numpy]()))
    gpt.model.encoder.layers[i].mlp.layers[3].b = Param(atype(weights["model.encoder.$(i-1).mlp.2.bias"][:cpu]()[:numpy]()))
end

gpt.model.cast_embed.w = Param(atype(weights["model.cast_embed.weight"][:cpu]()[:numpy]()))
gpt.model.cast_embed.b = Param(atype(weights["model.cast_embed.bias"][:cpu]()[:numpy]()))

# Decoder
gpt.model.latent_mixing.w = Param(atype(weights["model.latent_mixing.weight"][:cpu]()[:numpy]()))
gpt.model.latent_mixing.b = Param(atype(weights["model.latent_mixing.bias"][:cpu]()[:numpy]()))

for i in 1:model_config["n_layer"]
    gpt.model.decoder.layers[i].ln1.a = Param(atype(weights["model.decoder.$(i-1).ln1.weight"][:cpu]()[:numpy]()))
    gpt.model.decoder.layers[i].ln1.b = Param(atype(weights["model.decoder.$(i-1).ln1.bias"][:cpu]()[:numpy]()))
    gpt.model.decoder.layers[i].ln2.a = Param(atype(weights["model.decoder.$(i-1).ln2.weight"][:cpu]()[:numpy]()))
    gpt.model.decoder.layers[i].ln2.b = Param(atype(weights["model.decoder.$(i-1).ln2.bias"][:cpu]()[:numpy]()))

    gpt.model.decoder.layers[i].attn.key.w = Param(atype(weights["model.decoder.$(i-1).attn.key.weight"][:cpu]()[:numpy]()))
    gpt.model.decoder.layers[i].attn.key.b = Param(atype(weights["model.decoder.$(i-1).attn.key.bias"][:cpu]()[:numpy]()))
    gpt.model.decoder.layers[i].attn.query.w = Param(atype(weights["model.decoder.$(i-1).attn.query.weight"][:cpu]()[:numpy]()))
    gpt.model.decoder.layers[i].attn.query.b = Param(atype(weights["model.decoder.$(i-1).attn.query.bias"][:cpu]()[:numpy]()))
    gpt.model.decoder.layers[i].attn.value.w = Param(atype(weights["model.decoder.$(i-1).attn.value.weight"][:cpu]()[:numpy]()))
    gpt.model.decoder.layers[i].attn.value.b = Param(atype(weights["model.decoder.$(i-1).attn.value.bias"][:cpu]()[:numpy]()))
    gpt.model.decoder.layers[i].attn.proj.w = Param(atype(weights["model.decoder.$(i-1).attn.proj.weight"][:cpu]()[:numpy]()))
    gpt.model.decoder.layers[i].attn.proj.b = Param(atype(weights["model.decoder.$(i-1).attn.proj.bias"][:cpu]()[:numpy]()))

    gpt.model.decoder.layers[i].mlp.layers[1].w = Param(atype(weights["model.decoder.$(i-1).mlp.0.weight"][:cpu]()[:numpy]()))
    gpt.model.decoder.layers[i].mlp.layers[1].b = Param(atype(weights["model.decoder.$(i-1).mlp.0.bias"][:cpu]()[:numpy]()))
    gpt.model.decoder.layers[i].mlp.layers[3].w = Param(atype(weights["model.decoder.$(i-1).mlp.2.weight"][:cpu]()[:numpy]()))
    gpt.model.decoder.layers[i].mlp.layers[3].b = Param(atype(weights["model.decoder.$(i-1).mlp.2.bias"][:cpu]()[:numpy]()))
end

gpt.model.ln_f.a = Param(atype(weights["model.ln_f.weight"][:cpu]()[:numpy]()))
gpt.model.ln_f.b = Param(atype(weights["model.ln_f.bias"][:cpu]()[:numpy]()))

gpt.model.predict.w = Param(atype(weights["model.predict.weight"][:cpu]()[:numpy]()))
gpt.model.predict.b = Param(atype(weights["model.predict.bias"][:cpu]()[:numpy]()))

# codebook
gpt.model.codebook.embedding = Param(atype(weights["model.codebook.embedding"][:cpu]()[:numpy]()'))
gpt.model.codebook.ema_count = Param(atype(weights["model.codebook.ema_count"][:cpu]()[:numpy]()))
gpt.model.codebook.ema_w = Param(atype(weights["model.codebook.ema_w"][:cpu]()[:numpy]()'))

# padding vector
gpt.padding_vector = atype(normalize_joined_single(dataset, atype(zeros(gpt.transition_dim-1))));

# TransformerPrior Init and Model Loading

In [28]:
args = parser(super_args, experiment="train")
args["logbase"] = expanduser(args["logbase"])
args["savepath"] = expanduser(args["savepath"])
args["savepath"] = "/Users/mehmeteneserciyes/logs_julia/hopper-medium-replay-v2/T-1-42/"
block_size = args["subsampled_sequence_length"] ÷ args["latent_step"]
obs_dim = dataset.observation_dim

[ utils/setup ] Reading config: ../config/vqvae.jl:hopper_medium_replay_v2
/Users/mehmeteneserciyes/logs_julia/hopper-medium-replay-v2/T-1-42/ already exists. Proceeding...
Made directory /Users/mehmeteneserciyes/logs_julia/hopper-medium-replay-v2/T-1-42/


11

In [29]:
model_config = deepcopy(args)
model_config["block_size"] = block_size
model_config["observation_dim"] = obs_dim
model_config["n_embd"] = args["n_embd"] * args["n_head"]

512

In [30]:
# turn off dropout
model_config["embd_pdrop"] = 0.0f0
model_config["attn_pdrop"] = 0.0f0
model_config["resid_pdrop"] = 0.0f0

0.0f0

In [31]:
prior = TransformerPrior(model_config);

In [32]:
# encoder
prior.tok_emb = Param(atype(prior_weights["tok_emb.weight"][:cpu]()[:numpy]()'))
prior.pos_emb = Param(atype(permutedims(prior_weights["pos_emb"][:cpu]()[:numpy](), (3,2,1))))

prior.state_emb.w = Param(atype(prior_weights["state_emb.weight"][:cpu]()[:numpy]()))
prior.state_emb.b = Param(atype(prior_weights["state_emb.bias"][:cpu]()[:numpy]()))

for i in 1:model_config["n_layer"]
    prior.blocks.layers[i].ln1.a = Param(atype(prior_weights["blocks.$(i-1).ln1.weight"][:cpu]()[:numpy]()))
    prior.blocks.layers[i].ln1.b = Param(atype(prior_weights["blocks.$(i-1).ln1.bias"][:cpu]()[:numpy]()))
    prior.blocks.layers[i].ln2.a = Param(atype(prior_weights["blocks.$(i-1).ln2.weight"][:cpu]()[:numpy]()))
    prior.blocks.layers[i].ln2.b = Param(atype(prior_weights["blocks.$(i-1).ln2.bias"][:cpu]()[:numpy]()))

    prior.blocks.layers[i].attn.key.w = Param(atype(prior_weights["blocks.$(i-1).attn.key.weight"][:cpu]()[:numpy]()))
    prior.blocks.layers[i].attn.key.b = Param(atype(prior_weights["blocks.$(i-1).attn.key.bias"][:cpu]()[:numpy]()))
    prior.blocks.layers[i].attn.query.w = Param(atype(prior_weights["blocks.$(i-1).attn.query.weight"][:cpu]()[:numpy]()))
    prior.blocks.layers[i].attn.query.b = Param(atype(prior_weights["blocks.$(i-1).attn.query.bias"][:cpu]()[:numpy]()))
    prior.blocks.layers[i].attn.value.w = Param(atype(prior_weights["blocks.$(i-1).attn.value.weight"][:cpu]()[:numpy]()))
    prior.blocks.layers[i].attn.value.b = Param(atype(prior_weights["blocks.$(i-1).attn.value.bias"][:cpu]()[:numpy]()))
    prior.blocks.layers[i].attn.proj.w = Param(atype(prior_weights["blocks.$(i-1).attn.proj.weight"][:cpu]()[:numpy]()))
    prior.blocks.layers[i].attn.proj.b = Param(atype(prior_weights["blocks.$(i-1).attn.proj.bias"][:cpu]()[:numpy]()))

    prior.blocks.layers[i].mlp.layers[1].w = Param(atype(prior_weights["blocks.$(i-1).mlp.0.weight"][:cpu]()[:numpy]()))
    prior.blocks.layers[i].mlp.layers[1].b = Param(atype(prior_weights["blocks.$(i-1).mlp.0.bias"][:cpu]()[:numpy]()))
    prior.blocks.layers[i].mlp.layers[3].w = Param(atype(prior_weights["blocks.$(i-1).mlp.2.weight"][:cpu]()[:numpy]()))
    prior.blocks.layers[i].mlp.layers[3].b = Param(atype(prior_weights["blocks.$(i-1).mlp.2.bias"][:cpu]()[:numpy]()))
end

prior.ln_f.a = Param(atype(prior_weights["ln_f.weight"][:cpu]()[:numpy]()))
prior.ln_f.b = Param(atype(prior_weights["ln_f.bias"][:cpu]()[:numpy]()))
# no bias
prior.head.w = Param(atype(prior_weights["head.weight"][:cpu]()[:numpy]())); 

# Planning part

In [34]:
discount = dataset.discount
observation_dim = dataset.observation_dim
action_dim = dataset.action_dim

#######################
###### main loop ######
#######################
REWARD_DIM = VALUE_DIM = 1
transition_dim = observation_dim + action_dim + REWARD_DIM + VALUE_DIM

observation = env.reset()
total_reward = 0
discount_return = 0

0

In [35]:
observation

11-element Vector{Float64}:
  1.253712370089423
 -0.00038758963847066895
 -0.004536975813829556
  0.00418929445076901
 -0.002651174948780496
 -0.0006299949827345885
  0.0032714934892716318
  0.0007910854975872561
 -0.003877485202099127
  0.0045930625544841645
  0.00020891531807665565